# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 14 2022 3:23PM,253135,20,8898667,Else Nutrition,Released
1,Dec 14 2022 3:06PM,253134,10,SSR-000269,"Senseonics, Incorporated",Released
2,Dec 14 2022 2:59PM,253133,10,CTF0011543,"Citieffe, Inc.",Executing
3,Dec 14 2022 2:48PM,253132,21,636158,"NBTY Global, Inc.",Released
4,Dec 14 2022 2:48PM,253131,21,636157,"NBTY Global, Inc.",Executing
5,Dec 14 2022 2:43PM,253129,10,SO94464,"Senseonics, Incorporated",Released
6,Dec 14 2022 2:38PM,253128,10,SNAP1194329,"Snap Medical Industries, LLC",Released
7,Dec 14 2022 2:38PM,253128,10,SNAP1214221,"Snap Medical Industries, LLC",Released
8,Dec 14 2022 2:34PM,253127,19,ADV80009087,"AdvaGen Pharma, Ltd",Released
9,Dec 14 2022 2:34PM,253127,19,ADV80009095,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,253131,Executing,1
36,253132,Released,1
37,253133,Executing,1
38,253134,Released,1
39,253135,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253131,NaN,1.0,NaN
253132,NaN,NaN,1.0
253133,NaN,1.0,NaN
253134,NaN,NaN,1.0
253135,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253051,5.0,6.0,4.0
253053,14.0,4.0,7.0
253062,9.0,3.0,0.0
253064,0.0,0.0,23.0
253083,0.0,0.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253051,5,6,4
253053,14,4,7
253062,9,3,0
253064,0,0,23
253083,0,0,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253051,5,6,4
1,253053,14,4,7
2,253062,9,3,0
3,253064,0,0,23
4,253083,0,0,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253051,5,6,4
1,253053,14,4,7
2,253062,9,3,
3,253064,,,23
4,253083,,,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 14 2022 3:23PM,253135,20,Else Nutrition
1,Dec 14 2022 3:06PM,253134,10,"Senseonics, Incorporated"
2,Dec 14 2022 2:59PM,253133,10,"Citieffe, Inc."
3,Dec 14 2022 2:48PM,253132,21,"NBTY Global, Inc."
4,Dec 14 2022 2:48PM,253131,21,"NBTY Global, Inc."
5,Dec 14 2022 2:43PM,253129,10,"Senseonics, Incorporated"
6,Dec 14 2022 2:38PM,253128,10,"Snap Medical Industries, LLC"
8,Dec 14 2022 2:34PM,253127,19,"AdvaGen Pharma, Ltd"
24,Dec 14 2022 2:28PM,253126,19,"GUSA Granules USA, Inc."
25,Dec 14 2022 2:24PM,253125,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 14 2022 3:23PM,253135,20,Else Nutrition,,,1
1,Dec 14 2022 3:06PM,253134,10,"Senseonics, Incorporated",,,1
2,Dec 14 2022 2:59PM,253133,10,"Citieffe, Inc.",,1,
3,Dec 14 2022 2:48PM,253132,21,"NBTY Global, Inc.",,,1
4,Dec 14 2022 2:48PM,253131,21,"NBTY Global, Inc.",,1,
5,Dec 14 2022 2:43PM,253129,10,"Senseonics, Incorporated",,,1
6,Dec 14 2022 2:38PM,253128,10,"Snap Medical Industries, LLC",,,2
7,Dec 14 2022 2:34PM,253127,19,"AdvaGen Pharma, Ltd",,,16
8,Dec 14 2022 2:28PM,253126,19,"GUSA Granules USA, Inc.",,,1
9,Dec 14 2022 2:24PM,253125,16,Sartorius Bioprocess Solutions,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 3:23PM,253135,20,Else Nutrition,1,,
1,Dec 14 2022 3:06PM,253134,10,"Senseonics, Incorporated",1,,
2,Dec 14 2022 2:59PM,253133,10,"Citieffe, Inc.",,1,
3,Dec 14 2022 2:48PM,253132,21,"NBTY Global, Inc.",1,,
4,Dec 14 2022 2:48PM,253131,21,"NBTY Global, Inc.",,1,
5,Dec 14 2022 2:43PM,253129,10,"Senseonics, Incorporated",1,,
6,Dec 14 2022 2:38PM,253128,10,"Snap Medical Industries, LLC",2,,
7,Dec 14 2022 2:34PM,253127,19,"AdvaGen Pharma, Ltd",16,,
8,Dec 14 2022 2:28PM,253126,19,"GUSA Granules USA, Inc.",1,,
9,Dec 14 2022 2:24PM,253125,16,Sartorius Bioprocess Solutions,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 3:23PM,253135,20,Else Nutrition,1,,
1,Dec 14 2022 3:06PM,253134,10,"Senseonics, Incorporated",1,,
2,Dec 14 2022 2:59PM,253133,10,"Citieffe, Inc.",,1,
3,Dec 14 2022 2:48PM,253132,21,"NBTY Global, Inc.",1,,
4,Dec 14 2022 2:48PM,253131,21,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 3:23PM,253135,20,Else Nutrition,1.0,NaN,NaN
1,Dec 14 2022 3:06PM,253134,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Dec 14 2022 2:59PM,253133,10,"Citieffe, Inc.",NaN,1.0,NaN
3,Dec 14 2022 2:48PM,253132,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Dec 14 2022 2:48PM,253131,21,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 3:23PM,253135,20,Else Nutrition,1.0,0.0,0.0
1,Dec 14 2022 3:06PM,253134,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Dec 14 2022 2:59PM,253133,10,"Citieffe, Inc.",0.0,1.0,0.0
3,Dec 14 2022 2:48PM,253132,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Dec 14 2022 2:48PM,253131,21,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3543479,74.0,10.0,13.0
15,1518509,58.0,10.0,19.0
16,253125,1.0,0.0,0.0
19,1771743,25.0,0.0,0.0
20,759340,3.0,0.0,0.0
21,506263,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3543479,74.0,10.0,13.0
1,15,1518509,58.0,10.0,19.0
2,16,253125,1.0,0.0,0.0
3,19,1771743,25.0,0.0,0.0
4,20,759340,3.0,0.0,0.0
5,21,506263,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,74.0,10.0,13.0
1,15,58.0,10.0,19.0
2,16,1.0,0.0,0.0
3,19,25.0,0.0,0.0
4,20,3.0,0.0,0.0
5,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,74.0
1,15,Released,58.0
2,16,Released,1.0
3,19,Released,25.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,13.0,19.0,0.0,0.0,0.0,0.0
Executing,10.0,10.0,0.0,0.0,0.0,1.0
Released,74.0,58.0,1.0,25.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,13.0,19.0,0.0,0.0,0.0,0.0
1,Executing,10.0,10.0,0.0,0.0,0.0,1.0
2,Released,74.0,58.0,1.0,25.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,13.0,19.0,0.0,0.0,0.0,0.0
1,Executing,10.0,10.0,0.0,0.0,0.0,1.0
2,Released,74.0,58.0,1.0,25.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()